In [1]:
from matplotlib_venn import venn2
from scipy import stats
from collections import defaultdict
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import argparse
import seaborn as sns
import os
import math
import pandas as pd

COVERAGE_MAP = {
    "ChocoPy": ["chocopy.reference.semantic"],
    "Gson": ["com.google.gson"],
    "Jackson": ["com.fasterxml.jackson.databind.json", "com.fasterxml.jackson.core.json"],
    "Tomcat": ["org.apache.tomcat.util.descriptor", "org.apache.tomcat.util.digester"],
    "Closure": ["com.google.javascript.jscomp"]
}

TECHS = [("zest", "Zest"), 
          ("mutate", "Mu2"), 
          ("mutate-time-constrained", "Mu2"),
          ("mutate-no-save", "Mu2NoSave")]

TECH_NAMES = ["Zest", "Zest*", "Mu2 (fixed trials)", "Mu2 (fixed time)"]

In [2]:
def get_killed_mutants(file_name, exp_num, alg, mutant_killed_map, fuzz_results_dir):
    with open(file_name, "r") as f:
        lines = f.readlines()

    i = 0
    killed_mutants = set()
    seen_mutants = set()
    while i < len(lines):
        if "Running Mutant" in lines[i]:
            killed = False
            mutant_name = lines[i][lines[i].index("Mutant"):].strip()
            seen_mutants.add(mutant_name)
            i += 1
            while i < len(lines) and "Running Mutant" not in lines[i]:
                if "FAILURE" in lines[i]:
                    input_name = lines[i].split("input")[1].strip()
                    input_num = input_name[:input_name.index(":")]
                    killed = True
                i += 1
        if killed:
            killed_mutants.add(mutant_name)
            input_name = "id_" + '0' * (6 - len(input_num)) + input_num + ".0"
            input_file_name = os.path.join(fuzz_results_dir, "tmp%s/exp_%d/args_corpus/%s" % (alg, exp_num, input_name))
            mutant_killed_map[mutant_name].append(input_file_name)
    return killed_mutants, seen_mutants

def get_unique(zest_killed, mutate_killed):

    zest_uniq = zest_killed - mutate_killed
    mutate_uniq = mutate_killed - zest_killed
    both_found = zest_killed.intersection(mutate_killed)

    return zest_uniq, mutate_uniq, both_found

def get_coverage(cov_file, target_name):
    cov_df = pd.read_csv(cov_file)
    cov_df = cov_df[cov_df["PACKAGE"].apply(lambda x: any(y in x for y in COVERAGE_MAP[target_name]))]
    cov_df["BRANCH_TOTAL"] = cov_df["BRANCH_COVERED"] + cov_df["BRANCH_MISSED"]
    cov_df = cov_df[["PACKAGE", "BRANCH_COVERED", "BRANCH_TOTAL"]]
    return cov_df
    
    

def display_mutants(mutant_list, url, mutant_killed_map, display_inputs=False):
    for mutant in sorted(mutant_list):
        inputs = mutant_killed_map[mutant]
        line = mutant.split(":")[-1][:-1]
        filename = mutant[mutant.index("(") + 1:mutant.index(".java:") + 5]
        line_url = url + filename + "#" + line
        display_markdown("[{}]({})".format(mutant, line_url))
        if display_inputs:
            for input_file in mutant_killed_map[mutant]:
                with open(input_file, "r") as f:
                    # print(input_file)
                    print(f.read() + "\n--------------------------------------------------------------------\n")

def display_markdown(string):
    display(Markdown(string))
    
def display_comparison_table(union1, union2, intersect1, intersect2, tech1, tech2, url, mutant_killed_map, display_muts=False):
        
    table = "| Any {tech1}, No {tech2} | Any {tech2}, No {tech1} | All {tech1}, No {tech2} | All {tech2}, No {tech1} | \n | --- | --- | --- | --- | \n | {count1} | {count2} | {count3} | {count4} |".format(
        tech1=tech1,
        tech2=tech2,
        count1=len(union1 - union2),
        count2=len(union2 - union1),
        count3=len(intersect1 - union2),
        count4=len(intersect2 - union1))
    display_markdown(table)
    
    if display_muts:
        display_markdown("Mutants killed by any {tech1} and not any {tech2}:".format(tech1=tech1, tech2=tech2))
        display_mutants(list(union1 - union2), url, mutant_killed_map, False)
        display_markdown("Mutants killed by all {tech1} and not any {tech2}:".format(tech1=tech1, tech2=tech2))
        display_mutants(list(intersect1 - union2), url, mutant_killed_map, False)
        display_markdown("Mutants killed by any {tech2} and not any {tech1}:".format(tech1=tech1, tech2=tech2))
        display_mutants(list(union2 - union1), url, mutant_killed_map, False)
        display_markdown("Mutants killed by all {tech2} and not any {tech1}:".format(tech1=tech1, tech2=tech2))
        display_mutants(list(intersect2 - union1), url, mutant_killed_map, False)
    
def plot_killed_freq(list_sets, list_techs, target_name, filter_equal=False):
    
    all_killed_mutants = sorted(list(set.union(*(sum(list_sets, [])))))
    num_experiments = len(list_sets[0])
    list_set_frequency = []
    list_set_filtered = []
    list_hist = []
    
    for s in list_sets:
        
        set_killed_freq = [sum([x in killed_set for killed_set in s]) / num_experiments for x in all_killed_mutants]
        list_set_frequency.append(set_killed_freq)

    indices = [i for i in range(len(all_killed_mutants)) if any([s[i] < 1 for s in list_set_frequency])]
    all_filtered = [all_killed_mutants[i] for i in indices]
    
    for s in list_set_frequency:
        list_set_filtered.append(np.array([s[i] for i in indices]))

    freq_df = pd.DataFrame(
        dict(zip(list_techs, list_set_filtered))
    )
    

    sns.set(rc = {'figure.figsize':(10,8)})

    sns.set(font_scale=1.8)
    sns.violinplot(data=freq_df)
    plt.title("%s Killed Mutants" % target_name, fontdict={'fontsize': 24})
    plt.xlabel("Technique", fontdict={'fontsize': 20})
    plt.ylabel("Percentage Killed", fontdict={'fontsize': 20})
    plt.ylim(0, 1)
    plt.tight_layout()

    # plt.show()
    # plt.savefig("violin.pdf")
    
    diff_filtered1 = list_set_filtered[2] - list_set_filtered[0]
    diff_filtered2 = list_set_filtered[3] - list_set_filtered[0]
    diff_filtered3 = list_set_filtered[2] - list_set_filtered[1]
    display_markdown("Average Mu2 percent improvement over Zest baseline over %d mutants: %f" % (len(all_filtered), np.mean(diff_filtered1)))
    display_markdown("Average Mu2 (time-constrained) percent improvement over Zest baseline over %d mutants: %f" % (len(all_filtered), np.mean(diff_filtered2)))
    display_markdown("Average Mu2 percent improvement over Mu2NoSave baseline over %d mutants: %f" % (len(all_filtered), np.mean(diff_filtered3)))

    display_markdown("MWU test between Zest and Mu2: pvalue=%f" % stats.mannwhitneyu(list_set_filtered[0], list_set_filtered[2]).pvalue)
    display_markdown("MWU test between Zest and Mu2 time constrained: pvalue=%f" % stats.mannwhitneyu(list_set_filtered[0], list_set_filtered[3]).pvalue)
    display_markdown("MWU test between Mu2 and Mu2NoSave: pvalue=%f" % stats.mannwhitneyu(list_set_filtered[1], list_set_filtered[2]).pvalue)


def get_coverage_df(list_techs, list_coverage, target_name):
    cov_df = pd.DataFrame(
        dict(zip(list_techs, list_coverage))
    )
    rows = []
    zest_cov_mean = list_coverage[list_techs.index("Zest")].mean()
    for i in range(len(list_coverage)):
        if list_techs[i] == "Zest*":
            continue
        for j in range(len(list_coverage[i])):
            rows.append([j, list_techs[i], target_name, list_coverage[i][j]])
            # rows.append([j, list_techs[i], target_name, list_coverage[i][j] * 1.0 / zest_cov_mean])
    cov_df = pd.DataFrame(rows, columns=["Experiment", "Technique", "Target", "Coverage"])

    display_markdown("T-test between Zest and Mu2 coverage: pvalue=%f" % stats.ttest_ind(list_coverage[0], list_coverage[2]).pvalue)
    display_markdown("T-test between Zest and Mu2 time constrained coverage: pvalue=%f" % stats.ttest_ind(list_coverage[0], list_coverage[3]).pvalue)
    display_markdown("T-test between Mu2 and Mu2NoSave coverage: pvalue=%f" % stats.ttest_ind(list_coverage[1], list_coverage[2]).pvalue)
    
    for i in range(len(list_techs)):
        if list_techs[i] == "Zest*":
            continue
        print("%s: %f +- %f" % (list_techs[i], np.mean(list_coverage[i]), np.std(list_coverage[i])))
    return cov_df

In [8]:
def display_benchmark(target_name, filters_dir, fuzz_results_dir, cov_dir, url, num_experiments):
    mutant_killed_map = defaultdict(list)
    
    display_markdown("## {} Mu2 Experiments".format(target_name))
            
    union_all = {}
    union_seen_all = {}
    intersection_all = {}
    killed_all = {}
    num_killed_all = {}
    
    coverage_all = {}

    for tech, tech_dir in TECHS:
        
        all_killed_mutants = []
        all_seen_mutants = []
        total_cov = []
    
        for i in range(1, num_experiments + 1):
            killed_mutants, seen_mutants = get_killed_mutants(os.path.join(filters_dir, "{}-filter-{}.txt".format(tech, i)), i, tech_dir, mutant_killed_map, fuzz_results_dir)
            all_killed_mutants.append(killed_mutants)
            all_seen_mutants.append(seen_mutants)
            cov = get_coverage(os.path.join(cov_dir, "{}-coverage-{}.csv".format(tech, i)), target_name)
            total_cov.append(cov["BRANCH_COVERED"].sum())

        
        num_killed_mutants= [len(x) for x in all_killed_mutants]
        killed_stats = stats.describe(num_killed_mutants)
        # display_markdown("**{} Mutant Killed Summary**: ".format(tech))
        # display_markdown("Experiments: %d" % killed_stats.nobs)
        # display_markdown("Mean: %f" % killed_stats.mean)
        # display_markdown("Stddev: %f" % np.sqrt(killed_stats.variance))
                                           
        union_all[tech] = set.union(*all_killed_mutants)
        intersection_all[tech] = set.intersection(*all_killed_mutants)
        coverage_all[tech] = total_cov
        killed_all[tech] = all_killed_mutants
        union_seen_all[tech] = set.union(*all_seen_mutants) 
        num_killed_all[tech] = num_killed_mutants
    
#     display_markdown("T-test between Zest and Mu2: pvalue=%f" % stats.ttest_ind(num_killed_all["zest"], num_killed_all["mutate"]).pvalue)
#     display_markdown("T-test between Zest and Mu2 time constrained: pvalue=%f" % stats.ttest_ind(num_killed_all["zest"], num_killed_all["mutate-time-constrained"]).pvalue)
#     display_markdown("T-test between Mu2 and Mu2NoSave: pvalue=%f" % stats.ttest_ind(num_killed_all["mutate"], num_killed_all["mutate-no-save"]).pvalue)

#     display_markdown("**Mutant Killing Frequency**")

#     plot_killed_freq([killed_all["zest"], killed_all["mutate-no-save"], killed_all["mutate"], killed_all["mutate-time-constrained"]], TECH_NAMES, target_name)

    
    techs, coverages = zip(*coverage_all.items())
    cov_df = get_coverage_df(TECH_NAMES, np.array([coverage_all["zest"], coverage_all["mutate-no-save"], coverage_all["mutate"], coverage_all["mutate-time-constrained"]]), target_name)
    cov_df["Target"] = target_name
    display_markdown("**Mutant Comparison Table Zest vs Mu2**(same trials): ")
    
    display_markdown("Total seen mutants: %d" % len(set.union(union_seen_all["zest"], union_seen_all["mutate"])))
    display_markdown("Total killed mutants: %d" % len(set.union(union_all["zest"], union_all["mutate"])))

                               
    display_comparison_table(union_all["zest"], union_all["mutate"], intersection_all["zest"], intersection_all["mutate"], "Zest", "Mu2",
                            url, mutant_killed_map, True)

    display_markdown("**Mutant Comparison Table Zest vs Mu2Time**(same time): ")
    display_comparison_table(union_all["zest"], union_all["mutate-time-constrained"], intersection_all["zest"], intersection_all["mutate-time-constrained"], "Zest", "Mu2",
                            url, mutant_killed_map, False)    

    display_markdown("Total seen mutants: %d" % len(set.union(union_seen_all["zest"], union_seen_all["mutate-time-constrained"])))
    display_markdown("Total killed mutants: %d" % len(set.union(union_all["zest"], union_all["mutate-time-constrained"])))
    
#     display_markdown("**Mutant Comparison Table Zest\* vs Mu2Time**(same time): ")
                               
#     display_comparison_table(union_all["mutate-time-constrained"], union_all["mutate-no-save"], intersection_all["mutate-time-constrained"], intersection_all["mutate-no-save"], "Mu2", "Mu2NoSave",
#                             url, mutant_killed_map, False)
    
    display_markdown("**Mutant Comparison Table Zest\* vs Mu2**(same trial): ")
                               
    display_comparison_table(union_all["mutate"], union_all["mutate-no-save"], intersection_all["mutate"], intersection_all["mutate-no-save"], "Mu2", "Mu2NoSave",
                            url, mutant_killed_map, False)

    return cov_df

In [9]:
results_dir = "/home/vasumv/mu2_results/three_hours"
cov_dfs = []

display_markdown("# Mu2 Three Hour Experiments")
cov_dfs.append(display_benchmark("ChocoPy", os.path.join(results_dir, "filters/ChocoPy-filters/"), 
                           os.path.join(results_dir, "fuzz-results/ChocoPy-fuzz-results/"),
                           os.path.join(results_dir, "coverage/ChocoPy-coverage/"),
                           "https://github.com/chocopy/chocopy/tree/master/src/main/java/chocopy/reference/semantic/", 
                           20))
cov_dfs.append(display_benchmark("Gson", os.path.join(results_dir, "filters/gson-filters/"),
                           os.path.join(results_dir, "fuzz-results/gson-fuzz-results/"),
                           os.path.join(results_dir, "coverage/gson-coverage/"),
                           "https://github.com/google/gson/blob/gson-parent-2.8.9/gson/src/main/java/com/google/gson/stream/", 
                           20))
cov_dfs.append(display_benchmark("Jackson", os.path.join(results_dir, "filters/jacksonjson-filters/"),
                           os.path.join(results_dir, "fuzz-results/jacksonjson-fuzz-results/"),
                           os.path.join(results_dir, "coverage/jacksonjson-coverage/"),
                           "https://github.com/FasterXML/jackson-core/blob/jackson-core-2.13.0/src/main/java/com/fasterxml/jackson/core/json/", 20))
cov_dfs.append(display_benchmark("Tomcat", os.path.join(results_dir, "filters/apachetomcat-filters/"),
                           os.path.join(results_dir, "fuzz-results/apachetomcat-fuzz-results/"),
                           os.path.join(results_dir, "coverage/apachetomcat-coverage/"),
                           "https://github.com/FasterXML/jackson-core/blob/jackson-core-2.13.0/src/main/java/com/fasterxml/jackson/core/json/", 10))
cov_dfs.append(display_benchmark("Closure", os.path.join(results_dir, "filters/closure-filters/"),
                           os.path.join(results_dir, "fuzz-results/closure-fuzz-results/"),
                           os.path.join(results_dir, "coverage/closure-coverage/"),
                           "https://github.com/google/closure-compiler/blob/v20220202/src/com/google/javascript/jscomp/", 10))

# Mu2 Three Hour Experiments

## ChocoPy Mu2 Experiments

T-test between Zest and Mu2 coverage: pvalue=0.037856

T-test between Zest and Mu2 time constrained coverage: pvalue=0.080499

T-test between Mu2 and Mu2NoSave coverage: pvalue=0.428733

...

| Any Zest, No Mu2 | Any Mu2, No Zest | All Zest, No Mu2 | All Mu2, No Zest | 
 | --- | --- | --- | --- | 
 | 9 | 0 | 0 | 0 |

Total seen mutants: 503

Total killed mutants: 246

**Mutant Comparison Table Zest\* vs Mu2**(same trial): 

| Any Mu2, No Mu2NoSave | Any Mu2NoSave, No Mu2 | All Mu2, No Mu2NoSave | All Mu2NoSave, No Mu2 | 
 | --- | --- | --- | --- | 
 | 0 | 1 | 0 | 0 |

In [5]:
import itertools
normalized_cov = total_cov_df.copy()
zest_cov = total_cov_df[total_cov_df["Technique"] == "Zest"]
denom = []

def normalize(x, denom):
    return x / denom 

for i, target in enumerate(COVERAGE_MAP):
    normalized = zest_cov[zest_cov["Target"] == target]["Coverage"].mean()
    denom.append(normalized)
    normalized_cov["Coverage"] = normalized_cov.apply(lambda row: normalize(row["Coverage"], normalized) if row["Target"] == target else row["Coverage"], axis=1)
    normalized_cov["Target"] = normalized_cov["Target"].apply(lambda x: "%s\n(%d)" % (x, round(denom[i])) if x == target else x)

fig = plt.figure()
sns.set(font_scale=1.3)
ax = sns.barplot(data=normalized_cov, x="Target", y="Coverage", hue="Technique")
ax.set_title("Branch Coverage")
ax.set_ylabel("Coverage Relative to Zest")
hatches = itertools.cycle([' ', '//', '+'])
for i, bar in enumerate(ax.patches):
    if i % 5 == 0:
        hatch = next(hatches)
        bar.set_hatch(hatch)
    bar.set_hatch(hatch)
    
sns.set(font_scale=1.3)
ax.legend(fancybox=True, shadow=True)

def show_values_on_bars(axs):
    def _show_on_single_plot(ax):        
        for i in range(len(ax.patches)):
            p = ax.patches[i]
            if i < 5:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() + 0.03
                value = '{:.2f}'.format(p.get_height())
                ax.text(_x, _y, denom[i], ha="center") 

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)
sns.set(font_scale=0.8)

fig.savefig("branch_cov.pdf", bbox_inches='tight')

NameError: name 'total_cov_df' is not defined

In [ ]:
zest_cov = total_cov_df[total_cov_df["Technique"] == "Zest"]
for target in COVERAGE_MAP:
    print(zest_cov[zest_cov["Target"] == target]["Coverage"].mean())